<a href="https://colab.research.google.com/github/vivekvision/PySparkMLRepo/blob/main/PySparkLinearRegSatandardExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 28 kB/s 
     |████████████████████████████████| 198 kB 59.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=6b9a13c348942a24de02f90cb9cb91705ad3a712a461312f4c6791b441a65364
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [4]:
spark

In [5]:
from pyspark.ml.regression import LinearRegression

In [6]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [7]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [8]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')
lrModel = lr.fit(training)


In [9]:
lrModel.intercept

0.14228558260358093

In [10]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [12]:
training_summary = lrModel.summary

In [13]:
# Display R-Square 
training_summary.r2

0.027839179518600154

In [14]:
training_summary.rootMeanSquaredError

10.16309157133015

In [15]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [17]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [18]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                335|
|   mean| 0.4191189486108615|
| stddev| 10.228520749441586|
|    min|-26.805483428483072|
|    max|  27.78383192005107|
+-------+-------------------+



In [19]:
test_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 166|
|   mean|-0.07050330039125352|
| stddev|  10.519475509230222|
|    min| -28.571478869743427|
|    max|  27.111027963108548|
+-------+--------------------+



In [20]:
model = lr.fit(train_data)

In [21]:
test_results = model.evaluate(test_data)

In [22]:
test_results.residuals.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-------------------+
|          residuals|
+-------------------+
|-26.785795977706627|
|-26.439971138306166|
| -21.95880977234844|
|-21.239830597550704|
|-21.399272119038404|
| -19.19673459536863|
|-19.947785280699836|
| -20.52686855774079|
|  -19.2463834941745|
|-15.903489460315273|
|-19.472133063202904|
|-19.212777153298646|
|-14.689555851160206|
|-15.681449104883981|
|-15.326577951524506|
|-18.929276515402954|
|-13.581661416914546|
|-14.479869194616164|
|-12.753893207987593|
| -12.73735537431409|
+-------------------+
only showing top 20 rows



In [23]:
test_results.rootMeanSquaredError

10.834834943855073

In [24]:
# predict on new data

unlabeled_data = test_data.select('features')

In [25]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [26]:
predictions = model.transform(unlabeled_data)

In [27]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|    -1.7856828920368|
|(10,[0,1,2,3,4,5,...| -1.6060468994704657|
|(10,[0,1,2,3,4,5,...|  0.5264220081826368|
|(10,[0,1,2,3,4,5,...|   1.182347981761493|
|(10,[0,1,2,3,4,5,...|  1.5147113447649794|
|(10,[0,1,2,3,4,5,...| -0.6762564426997767|
|(10,[0,1,2,3,4,5,...| 0.16502249108529854|
|(10,[0,1,2,3,4,5,...|  0.8595499423690693|
|(10,[0,1,2,3,4,5,...|  -0.155952536040055|
|(10,[0,1,2,3,4,5,...| -0.8156073732898166|
|(10,[0,1,2,3,4,5,...|  2.7799260418917964|
|(10,[0,1,2,3,4,5,...|   2.951346877845916|
|(10,[0,1,2,3,4,5,...| -1.2619567146343673|
|(10,[0,1,2,3,4,5,...|-0.05063916735526408|
|(10,[0,1,2,3,4,5,...|-0.11080684190671075|
|(10,[0,1,2,3,4,5,...|   3.594509035480612|
|(10,[0,1,2,3,4,5,...|  -1.181096836016581|
|(10,[0,1,2,3,4,5,...|  0.5037382634634608|
|(10,[0,1,2,3,4,5,...| -0.6667015679031644|
|(10,[0,1,2,3,4,5,...|-0.3025726